# Task 1: Model Training

## 1. Set Up the Environment:

- Install necessary libraries such as PyTorch and torchvision.
- Import required packages

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import os

## 2. Download and Prepare the CIFAR-10 Dataset:

- Download the CIFAR-10 dataset using torchvision.datasets.
- Split the dataset into training (40,000 images) and validation (10,000 images) sets.

In [ ]:
# Check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Download the CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_size = int(0.8 * len(trainset))   # training split : 40,000 images
val_size = len(trainset) - train_size  # validation split : 10,000 images
train_dataset, val_dataset = torch.utils.data.random_split(trainset, [train_size, val_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=100,
                                          shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=100,
                                         shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,shuffle=False, num_workers=2)

Using device: cuda:0


100%|██████████| 170498071/170498071 [00:18<00:00, 9212764.73it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## 3. Define the CNN Model:
- Choose a CNN architecture (VGG-16).
- Modify the last layer to have 10 output classes for the CIFAR-10 dataset.

In [ ]:
net = models.vgg16(pretrained=True)
net.classifier[6] = nn.Linear(net.classifier[6].in_features, 10)
net = net.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 173MB/s]


## 4. Define Loss Function and Optimizer:

- Use CrossEntropyLoss and an optimizer like SGD

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## 5. Train the Model:

- Train the model for 10 epochs and evaluate on the validation set.

In [ ]:
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:  # print every 100 mini-batches
            print(f'[Epoch: {epoch + 1}, Batch: {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

    # Evaluate the model on the validation set after each epoch
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    epoch_accuracy = 100 * correct / total
    print(f'Accuracy on validation images after epoch {epoch + 1}: {epoch_accuracy:.2f}%')

print('Finished Training')


[Epoch: 1, Batch: 100] loss: 1.300
[Epoch: 1, Batch: 200] loss: 0.782
[Epoch: 1, Batch: 300] loss: 0.702
[Epoch: 1, Batch: 400] loss: 0.629
Accuracy on validation images after epoch 1: 79.13%
[Epoch: 2, Batch: 100] loss: 0.514
[Epoch: 2, Batch: 200] loss: 0.501
[Epoch: 2, Batch: 300] loss: 0.482
[Epoch: 2, Batch: 400] loss: 0.453
Accuracy on validation images after epoch 2: 82.91%
[Epoch: 3, Batch: 100] loss: 0.370
[Epoch: 3, Batch: 200] loss: 0.360
[Epoch: 3, Batch: 300] loss: 0.367
[Epoch: 3, Batch: 400] loss: 0.369
Accuracy on validation images after epoch 3: 83.65%
[Epoch: 4, Batch: 100] loss: 0.293


In [ ]:
# Evaluate the trained model on the validation dataset and report the accuracy achieved
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct / total}%')


Accuracy of the network on the test images: 85.85%


# Task 2: Model Pruning


## 1. Apply Pruning Techniques & Evaluate Pruned Models:

- Use PyTorch's pruning functionalities to prune the model.
- Experiment with different pruning ratios.
- Evaluate the pruned models on the validation set

In [ ]:
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import numpy as np

In [ ]:
# Function to apply pruning to each layer in the model
def prune_model(model, amount):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)

In [ ]:
# Function to remove pruning re-parametrization
def remove_pruning(model):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            prune.remove(module, 'weight')

In [ ]:
# Evaluate model function
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    model.train()
    return 100 * correct / total

In [ ]:
  # Experiment with 20 different pruning ratios and evaluate the model
ratios = np.linspace(0.30, 0.95, 20)  # 20 values from 30% to 95% pruning
results = {}

In [ ]:
for ratio in ratios:
    model_copy = models.vgg16(pretrained=True)
    model_copy.classifier[6] = nn.Linear(model_copy.classifier[6].in_features, 10)
    model_copy.load_state_dict(net.state_dict())
    model_copy.to(device)

    # Apply pruning
    prune_model(model_copy, ratio)

    # Evaluate pruned model on validation set
    val_accuracy = evaluate_model(model_copy, valloader)
    print(f'Pruning ratio {ratio:.2f}: Accuracy on validation images: {val_accuracy:.2f}%')

    # Evaluate pruned model on test set
    test_accuracy = evaluate_model(model_copy, testloader)
    print(f'Pruning ratio {ratio:.2f}: Accuracy on test images: {test_accuracy:.2f}%')

    results[ratio] = (val_accuracy, test_accuracy)

    # Save the pruned model with a unique name
    pruned_model_filename = f'pruned_vgg16_cifar10_{ratio:.2f}.pth'
    torch.save(model_copy.state_dict(), pruned_model_filename)

    # Remove pruning re-parametrization
    remove_pruning(model_copy)

Pruning ratio 0.30: Accuracy on validation images: 86.00%
Pruning ratio 0.30: Accuracy on test images: 86.04%
Pruning ratio 0.33: Accuracy on validation images: 86.03%
Pruning ratio 0.33: Accuracy on test images: 85.80%
Pruning ratio 0.37: Accuracy on validation images: 85.52%
Pruning ratio 0.37: Accuracy on test images: 85.58%
Pruning ratio 0.40: Accuracy on validation images: 85.42%
Pruning ratio 0.40: Accuracy on test images: 85.20%
Pruning ratio 0.44: Accuracy on validation images: 84.76%
Pruning ratio 0.44: Accuracy on test images: 84.58%
Pruning ratio 0.47: Accuracy on validation images: 84.26%
Pruning ratio 0.47: Accuracy on test images: 84.04%
Pruning ratio 0.51: Accuracy on validation images: 82.49%
Pruning ratio 0.51: Accuracy on test images: 82.72%
Pruning ratio 0.54: Accuracy on validation images: 80.38%
Pruning ratio 0.54: Accuracy on test images: 80.15%
Pruning ratio 0.57: Accuracy on validation images: 78.48%
Pruning ratio 0.57: Accuracy on test images: 78.54%
Pruning ra

**As pruning ratio is increasing the Accuracy is decreasing however it is maximum at 44% **

## 2.  Choose Best pruning ratio & Save the Model
- choose the best pruning ratio.
- Save the original and pruned models for future use.

In [ ]:
# Choose the best pruning ratio
best_ratio = max(results, key=results.get)
print(f'Best pruning ratio: {best_ratio:.2f} with accuracy: {results[best_ratio][0]:.2f}%')

# Save the best pruned model
best_pruned_model_filename = f'best_pruned_vgg16_cifar10_{best_ratio:.2f}.pth'
torch.save(model_copy.state_dict(), best_pruned_model_filename)
print(f'Saved the best pruned model as {best_pruned_model_filename}')

# Save the Original model
torch.save(net.state_dict(), 'model.pth')
print(f'Saved the original model as model.pth')

Best pruning ratio: 0.33 with accuracy: 86.03%
Saved the best pruned model as best_pruned_vgg16_cifar10_0.33.pth
Saved the original model as model.pth


## 3. Evaluate P50 and P90 Performance:
- evaluate the P50 and P90 of the model performance before and after pruning.


In [ ]:
# Evaluate the pruned model on p50, p90 performance
p50, p90 = np.percentile(list(results.values()), [50, 90])
print(f'p50 accuracy: {p50:.2f}%, p90 accuracy: {p90:.2f}%')

p50 accuracy: 73.09%, p90 accuracy: 85.60%


In [ ]:
from google.colab import files

files.download('model.pth')
files.download('pruned_vgg16_cifar10_0.40.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch

# Assuming 'net' is your trained model
model_path = '/content/model.pth'
pruned_model_path = '/content/pruned_vgg16_cifar10_0.40.pth'

torch.save(net.state_dict(), model_path)
torch.save(pruned_net.state_dict(), pruned_model_path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

NameError: name 'hi' is not defined

### Summary Report

#### Original Model
- Accuracy on Validation Dataset: 86.1%
- Accuracy on Test Dataset: 85.85%

#### Pruned Model
- Pruning Ratio: 0.4
- Accuracy on Validation Dataset: 85.42%
- Accuracy on Test Dataset: 85.2%
- Reduction in Model Size: 40%

#### P50 and P90 Performance

- Pruned Model: P50: 73.09%, P90: 85.6%

### Conclusion
The pruned model maintains an accuracy within 1% of the original model while reducing the model size by 40%.